In [1]:
import os
import sys
import re
import pandas as pd
import commands
import ast
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
from collections import Counter
import re
import numpy as np
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


In [2]:
sqlContext.sql("use dsc60230_gcct_tz_db")

DataFrame[]

In [17]:
sqlContext.tables("dsc60230_gcct_tz_db").where("tableName not like '%temptz%'").where("tableName not like '%dup%'").where("tableName not like '%bad%'").show(1000,False)

+-------------------+--------------------------------------+-----------+
|database           |tableName                             |isTemporary|
+-------------------+--------------------------------------+-----------+
|dsc60230_gcct_tz_db|account                               |false      |
|dsc60230_gcct_tz_db|activityparty                         |false      |
|dsc60230_gcct_tz_db|annotation                            |false      |
|dsc60230_gcct_tz_db|appointment                           |false      |
|dsc60230_gcct_tz_db|businessunit                          |false      |
|dsc60230_gcct_tz_db|calendar                              |false      |
|dsc60230_gcct_tz_db|calendarrule                          |false      |
|dsc60230_gcct_tz_db|contact                               |false      |
|dsc60230_gcct_tz_db|cxlvhlp_chatactivity                  |false      |
|dsc60230_gcct_tz_db|cxlvhlp_chatqueuestatistic            |false      |
|dsc60230_gcct_tz_db|cxlvhlp_surveyitem            

In [7]:
#print sorted columns of a table #
sqlContext.sql("desc calendarrule").sort("col_name").show(1000,False)

+----------------------------+--------------+-------+
|col_name                    |data_type     |comment|
+----------------------------+--------------+-------+
|calendarid                  |string        |null   |
|createdby                   |string        |null   |
|createdby_entitytype        |string        |null   |
|createdon                   |timestamp     |null   |
|description                 |string        |null   |
|duration                    |int           |null   |
|effectiveintervalend        |timestamp     |null   |
|effectiveintervalstart      |timestamp     |null   |
|effort                      |decimal(20,10)|null   |
|endtime                     |timestamp     |null   |
|entityid                    |string        |null   |
|entitystring                |string        |null   |
|eventdatetime               |string        |null   |
|eventtype                   |string        |null   |
|extentcode                  |decimal(20,10)|null   |
|gcct_calendarrule_identity 

In [5]:
#print tables with the number of columns
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%temptz%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            a=len(sqlCtx.table(k).columns)
            print k,a

account 356
activityparty 43
annotation 61
appointment 129
bad_record 9
businessunit 108
calendar 29
calendarrule 44
contact 430
cxlvhlp_chatactivity 177
cxlvhlp_chatqueuestatistic 61
cxlvhlp_surveyitem 58
email 160
fax 105
gcct_accountresponsibleagent 64
gcct_additionalsymptomcodes 74
gcct_addportalmessage 131
gcct_arbitrationclaimprocessing 265
gcct_buybackevaluationmilestones 165
gcct_caseassignment 134
gcct_caseclassification 120
gcct_casedispositiontype 78
gcct_coachback 128
gcct_country 59
gcct_customersatisfactiontools 151
gcct_delegationofauthority 90
gcct_demandltrtpsmclaimsprocessing 169
gcct_doaprogramcode 59
gcct_documentcustomerrecontact 125
gcct_engine 53
gcct_executive 62
gcct_executiveliaison 124
gcct_fieldinvolvementassistancerequest 164
gcct_fmcccontract 86
gcct_fulfillmentvendor 53
gcct_genericinformation 122
gcct_genericinformationtopic 67
gcct_generictopic 52
gcct_goodfaithreview 188
gcct_knownpartsdelay 71
gcct_lemonlawcriteria 63
gcct_loanercost 70
gcct_loyaltyas

In [11]:
#To search for a column in whole database
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%temptz%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            a=sqlCtx.table(k).columns
            a.sort()
            for n in a:
                if "accountid" == str(n).lower(): # approx match if "account" in str(n).lower():
                    print k,n

account accountid
contact accountid
incident accountid


In [6]:
u11=['gcct_customerinrental','gcct_vehicleoffroad','gcct_vhrreadydate','gcct_regardingid','gcct_regardingidname','msdyn_orgchangestatus','msdyn_orgchangestatusname','gcct_dealerportalrole','azureactivedirectoryobjectid','ccof_dealeremail','ccof_dealername','ccof_dealeraddresscity','ccof_buyer','ccof_buyername','ccof_iscommercialaccount','ccof_dealeraddressstate','ccof_buyeridtype','ccof_buyeryominame','ccof_dealeraddresspostalcode','ccof_dealeraddressstreet','ccof_iscommercialaccountname','ccof_jitexpirationdatetime','ccof_dealertelephone','ccof_dealerfax','prefix','filepointer','storagepointer']
#To search for a column in whole database
for t in u11:
    for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%temptz%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            a=sqlCtx.table(k).columns
            a.sort()
            for n in a:
                if t == str(n):
                    print k,n

gcct_loyaltyassistance gcct_customerinrental
gcct_partssmeassistancerequest gcct_customerinrental
gcct_rentaldetail gcct_customerinrental
gcct_technicalassistancerequest gcct_customerinrental
gcct_warrantyloanerrequest gcct_customerinrental
gcct_loyaltyassistance gcct_vehicleoffroad
gcct_partsorderstatus gcct_vehicleoffroad
gcct_partssmeassistancerequest gcct_vehicleoffroad
gcct_rentaldetail gcct_vehicleoffroad
gcct_technicalassistancerequest gcct_vehicleoffroad
gcct_vehicleoffroad gcct_vehicleoffroad
gcct_warrantyloanerrequest gcct_vehicleoffroad
systemuser azureactivedirectoryobjectid


In [3]:
#search columns --unique
#customized for codes filtering non-codenames
col_list=[]
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%bkp%'").where("tableName not like '%temptz%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            a=sqlCtx.table(k).columns
            a.sort()
            for n in a:
                if "code" in str(n):
                    if "name" not in str(n):
                          col_list.append(str(n))


len(set(col_list))
#for l in set(col_list):
#    print(l)

139

# search a message id in the database

In [ ]:
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%temptz%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            a=sqlCtx.table(k).filter("to_date(lastupdatedatetime)  = date_sub(CAST(current_timestamp() as DATE), 1)")
            b=a.filter("message__id = '08586509193684189898969403629CU04'")
            if b.count()>0:
                print("Message__id found in "+k)
            

In [ ]:
sqlContext.table("bad_record").filter("message__id='08586509309542064342314412696CU08'").show(100,False)

In [39]:
sqlCtx.sql("select gcct_vehicleid from gcct_vehicle ").distinct().where("gcct_vehicleid like '%2F%'").show(1000,False)

+------------------------------------+
|gcct_vehicleid                      |
+------------------------------------+
|E2FD48D8-0DE9-E811-A97E-000D3A37D634|
|93B18DF7-2FEF-E811-A969-000D3A328ED9|
|8C698D20-92F4-E811-A970-000D3A328908|
|C72F51AA-A3EC-E811-A968-000D3A34AE50|
|FA2F7DC2-EB96-E811-A83D-000D3A37CC26|
+------------------------------------+



# Bad Records 

In [12]:
sqlCtx.table('bad_record').filter("to_date(lastupdatedatetime) >= '2019-02-07'").groupby("datatype").count().show(100,False)

+--------------------+-----+
|datatype            |count|
+--------------------+-----+
|systemuser          |51   |
|account             |3159 |
|calendar            |28   |
|gcct_caseassignment |1    |
|annotation          |1105 |
|gcct_vehicleowner   |3    |
|contact             |120  |
|activityparty       |1    |
|gcct_vehicle        |928  |
|slakpiinstance      |1    |
|phonecall           |511  |
|incident            |20819|
|msdyn_survey        |28   |
|cxlvhlp_chatactivity|2    |
+--------------------+-----+



In [9]:
#Bad records due to data discripency
sqlCtx.sql('select datatype,date_format(lastupdatedatetime,"yyyyMM") as Year_Month,exception from bad_record where  exception like "%Datatype discrepancy%"').groupby("datatype","Year_Month").count().sort("datatype").show(1000,False)

+--------------------------+----------+-----+
|datatype                  |Year_Month|count|
+--------------------------+----------+-----+
|account                   |201812    |5    |
|contact                   |201902    |17033|
|contact                   |201812    |84   |
|contact                   |201901    |12   |
|gcct_loyaltyassistance    |201812    |249  |
|gcct_loyaltyassistance    |201901    |71   |
|gcct_warrantyloanerrequest|201901    |55   |
|gcct_warrantyloanerrequest|201812    |9    |
+--------------------------+----------+-----+



In [9]:
#look up for various reasons for bad record
sqlCtx.sql('select distinct reason from bad_record').show(1000,False)

+----------------------------------------------------------------+
|reason                                                          |
+----------------------------------------------------------------+
|Error Parsing gcctravprocessingmilestones                       |
|Error Parsing cxlvhlp_chatqueuestatistic                        |
|Error Parsing gcctbuybackevaluationmilestones                   |
|Error Parsing gccttestdriverequest                              |
|Error Parsing slakpiinstance                                    |
|Error Parsing                                                   |
|Error Parsing gcctwarrantyloanerrequest                         |
|Error Parsing msdynsurveyinvite                                 |
|Error Parsing gcct_goodfaithreview                              |
|Error Parsing activityparty                                     |
|Error Parsing gcct_vehicle                                      |
|Duplicate record. Rejected by the process                    

addn info where error parsing is classified into data discrepancy due to mismatching data types and empty messages 

In [5]:
sqlCtx.sql('select exception from bad_record where reason like "Error Parsing contact"').distinct().show(1000,False)

+---------------------------------------------------------------------------------+
|exception                                                                        |
+---------------------------------------------------------------------------------+
|Datatype discrepancy found for the column: gcct_bestdaytimephone	value: 0        |
|Datatype discrepancy found for the column: gcct_bestdaytimephone	value: 658030002|
|Datatype discrepancy found for the column: gcct_bestdaytimephone	value: 658030001|
|Datatype discrepancy found for the column: gcct_bestdaytimephone	value: 658030000|
|No Entity name Found                                                             |
+---------------------------------------------------------------------------------+



# New entity added without approval goes into bad records

Find the tables added based on bad record data type. No date filter

In [30]:
#.where("to_date(lastupdatedatetime)  = date_sub(current_date, 1)")
database="dsc60230_gcct_tz_db"
p1=sqlCtx.table("bad_record").select("datatype").distinct().toPandas()["datatype"]
p2=sqlContext.tables(database).select('tableName').where("tableName not like '%dup%'").where("tableName not like '%temptz%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").toPandas()["tableName"]
p3=list(set([i.encode('utf-8') for i in list(p1)]).difference(set([j.encode('utf-8') for j in list(p2)])))
print"Following tables are added"
for i in p3:
    print i


Following tables are added

gcctrentaldetail
gcctgoodfaithreview
gcctcaseassignment
gcctdemandltrtpsmclaimsprocessing
gcct_annotation
gcctvehicleclass
gccttestdriverequest
gccttechnicalassistancerequest
gcctpcarequest
test22
msdynsurveyinvite
gcctloyaltyassistance
gcctpartssmeassistancerequest
gcctgenericinformation
gcctvehicleowner
gcctexecutiveliaison
gcctbuybackevaluationmilestones
cxlvhlpchatqueuestatistic
gcctvehicleoffroad
gcctarbitrationclaimprocessing
gcctravprocessingmilestones
gcctvehicle
msdynsurvey
gcctvehicleloyaltyallowance
Entity
gcctloyaltycostdetails
gcctwarrantyloanerrequest
gcctcustomersatisfactiontools
gcctoffer
gcctfieldinvolvementassistancerequest


Find the tables added based on the error exception. No date filter

In [29]:
#.where("to_date(lastupdatedatetime)  = date_sub(current_date, 1)")
sqlCtx.table('bad_record').filter(" exception like '%not part of approved entities%'").groupby("datatype").count().sort("datatype").show(1000,False)

+--------------------------------------+-----+
|datatype                              |count|
+--------------------------------------+-----+
|activityparty                         |7431 |
|cxlvhlpchatqueuestatistic             |80   |
|gcct_addportalmessage                 |1    |
|gcct_annotation                       |60   |
|gcct_arbitrationclaimprocessing       |25   |
|gcct_buybackevaluationmilestones      |20   |
|gcct_caseclassification               |22   |
|gcct_customersatisfactiontools        |8    |
|gcct_demandltrtpsmclaimsprocessing    |20   |
|gcct_doaprogramcode                   |193  |
|gcct_executiveliaison                 |24   |
|gcct_fieldinvolvementassistancerequest|13   |
|gcct_genericinformation               |72   |
|gcct_goodfaithreview                  |20   |
|gcct_loyaltyassistance                |435  |
|gcct_offer                            |17   |
|gcct_partssmeassistancerequest        |29   |
|gcct_pcarequest                       |17   |
|gcct_ravproc

Example data where a table went into bad record since it was not included in schema

In [18]:
sqlCtx.table('bad_record').where(" exception like '%not part of approved entities%'").filter("datatype = 'msdynsurvey'").select("reason","exception").show(1000,False)

+-------------------------+------------------------------------------------------------------+
|reason                   |exception                                                         |
+-------------------------+------------------------------------------------------------------+
|Error Parsing msdynsurvey|The entity msdynsurvey not part of approved entities to be parsed.|
|Error Parsing msdynsurvey|The entity msdynsurvey not part of approved entities to be parsed.|
+-------------------------+------------------------------------------------------------------+



In [4]:
#add info with date
sqlCtx.sql('select datatype,date_format(lastupdatedatetime,"yyyyMM") as Year_Month,exception from bad_record where  exception like "%not part of approved entities%"').groupby("datatype","Year_Month").count().sort("datatype").show(1000,False)

+--------------------------------------+----------+-----+
|datatype                              |Year_Month|count|
+--------------------------------------+----------+-----+
|activityparty                         |201812    |718  |
|activityparty                         |201811    |6713 |
|cxlvhlpchatqueuestatistic             |201811    |64   |
|cxlvhlpchatqueuestatistic             |201812    |16   |
|gcct_addportalmessage                 |201811    |1    |
|gcct_annotation                       |201811    |31   |
|gcct_annotation                       |201812    |29   |
|gcct_arbitrationclaimprocessing       |201811    |25   |
|gcct_buybackevaluationmilestones      |201811    |20   |
|gcct_caseclassification               |201811    |22   |
|gcct_customersatisfactiontools        |201811    |6    |
|gcct_customersatisfactiontools        |201812    |2    |
|gcct_demandltrtpsmclaimsprocessing    |201812    |2    |
|gcct_demandltrtpsmclaimsprocessing    |201811    |18   |
|gcct_doaprogr

# Understanding datatype changes

Whenever there is a data type change, we must verify the existing data type.
In this example, two columns have been changed in a table

In [6]:
sqlCtx.table('gcct_partssmeassistancerequest').filter("to_date(lastupdatedatetime) >= '2018-02-07'").select("gcct_vehicleoffroad","gcct_customerinrental").distinct().show(100,False)

+-------------------+---------------------+
|gcct_vehicleoffroad|gcct_customerinrental|
+-------------------+---------------------+
|true               |false                |
|true               |true                 |
|false              |false                |
|false              |true                 |
+-------------------+---------------------+



 Next we verify the last time when the data was seen in DSC

In [7]:
sqlCtx.sql("select to_date(lastupdatedatetime) as date2 from gcct_partssmeassistancerequest").groupby("date2").count().show(100,False)

+----------+-----+
|date2     |count|
+----------+-----+
|2019-02-01|34   |
|2019-02-07|1    |
|2019-02-05|4    |
|2019-02-06|4    |
|2019-02-03|2    |
|2019-02-13|7    |
|2019-02-04|6    |
+----------+-----+



Now we search for the changed column in all the tables of databases

In [11]:
#To search for a column in whole database
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%temptz%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            a=sqlCtx.table(k).columns
            a.sort()
            for n in a:
                if "accountid" == str(n).lower(): # approx match if "account" in str(n).lower():
                    print k,n

account accountid
contact accountid
incident accountid


In case we get a list of columns that have changed data type

In [6]:
u11=['gcct_customerinrental','gcct_vehicleoffroad','gcct_vhrreadydate','gcct_regardingid','gcct_regardingidname','msdyn_orgchangestatus','msdyn_orgchangestatusname','gcct_dealerportalrole','azureactivedirectoryobjectid','ccof_dealeremail','ccof_dealername','ccof_dealeraddresscity','ccof_buyer','ccof_buyername','ccof_iscommercialaccount','ccof_dealeraddressstate','ccof_buyeridtype','ccof_buyeryominame','ccof_dealeraddresspostalcode','ccof_dealeraddressstreet','ccof_iscommercialaccountname','ccof_jitexpirationdatetime','ccof_dealertelephone','ccof_dealerfax','prefix','filepointer','storagepointer']
#To search for a column in whole database
for t in u11:
    for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%temptz%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            a=sqlCtx.table(k).columns
            a.sort()
            for n in a:
                if t == str(n):
                    print k,n

gcct_loyaltyassistance gcct_customerinrental
gcct_partssmeassistancerequest gcct_customerinrental
gcct_rentaldetail gcct_customerinrental
gcct_technicalassistancerequest gcct_customerinrental
gcct_warrantyloanerrequest gcct_customerinrental
gcct_loyaltyassistance gcct_vehicleoffroad
gcct_partsorderstatus gcct_vehicleoffroad
gcct_partssmeassistancerequest gcct_vehicleoffroad
gcct_rentaldetail gcct_vehicleoffroad
gcct_technicalassistancerequest gcct_vehicleoffroad
gcct_vehicleoffroad gcct_vehicleoffroad
gcct_warrantyloanerrequest gcct_vehicleoffroad
systemuser azureactivedirectoryobjectid


Using map to analyze each record of the table
This can be used when we deal with small amount of data, hence used at a daily monitoring

In [145]:
k="account"
if k=="account":
            #k=m.asDict().values()[0].encode('utf-8')
            v1=sqlCtx.table(k)

from collections import Counter
import re
def freq(lst):
    d = {}
    for i in lst:
        if d.get(i):
            d[i] += 1
        else:
            d[i] = 1
    return d

v1.rdd.map(lambda x: str(x).replace("Row(","").replace("u\'","").replace("\'","").replace("\\\\","").replace("{",",").replace("}","").replace(", ",",").replace("=",":").replace("\"","").replace("false","False").replace("true","True").replace("null","None")).map(lambda g: re.sub("datetime.datetime\(.*?\)", "", g)).map(lambda w: w.replace(")","")).take(2)

['account_identity:52,account_rowversion:AAAAAAAA1Xs:,id:None,sinkcreatedon:None,sinkmodifiedon:None,statecode:0,statuscode:1,customertypecode:None,address1_freighttermscode:None,address1_addresstypecode:None,accountclassificationcode:1,shippingmethodcode:1,preferredappointmenttimecode:None,gcct_preferredemail:None,address2_freighttermscode:1,accountcategorycode:None,paymenttermscode:None,businesstypecode:2,industrycode:None,gcct_fleetstatuscode:None,address1_shippingmethodcode:None,preferredappointmentdaycode:None,customersizecode:1,preferredcontactmethodcode:None,accountratingcode:1,gcct_profiletype:None,address2_addresstypecode:1,address2_shippingmethodcode:1,gcct_dealertype:None,ownershipcode:None,territorycode:1,gcct_accountstatus:None,gcct_sourcetype:None,gcct_isprimarypacode:None,gcct_marketingemailpreference:None,donotbulkpostalmail:False,donotpostalmail:False,gcct_isgenericaccount:None,gcct_marketingmailpreference:None,gcct_donotallowcall:False,gcct_isinmarket:None,gcct_haslik

# Nested data

In [10]:
k="account"
if k=="account":
            v1=sqlCtx.table(k).filter("to_date(lastupdatedatetime)  = date_sub(current_date, 1)") 
            col_list=set(v1.columns)
            v2=v1.rdd.map(lambda x: str(x).replace("Row(","").replace("u\'","").replace("\'","").replace("\\\\","").replace("\"","")).map(lambda z: re.findall(r',([a-zA-Z]+):{', z)).filter(lambda x: len(x) > 0).collect()
            v3=set([item for sublist in v2 for item in sublist])
            print k,v3

account set([])


In [9]:
#Find nested data 
#searches for pattern ,*.*:{ in every line of every table 

for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").where("tableName not like '%dup%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            v1=sqlCtx.table(k).filter("to_date(lastupdatedatetime)  = date_sub(current_date, 1)") 
            #print(k)
            v3={}
            v2=v1.rdd.map(lambda x: str(x).replace("Row(","").replace("u\'","").replace("\'","").replace("\\\\","").replace("\"","")).map(lambda z: re.findall(r',([a-zA-Z]+):{', z)).filter(lambda x: len(x) > 0).collect()
            v3=set([item for sublist in v2 for item in sublist])
            if len(v3) >0:
                print k,v3
            

cxlvhlp_chatactivity set(['customer', 'mobile', 'authenticated', 'activityadditionalparams', 'chat'])


{'account_identity',
 'account_rowversion',
 'accountcategorycode',
 'accountclassificationcode',
 'accountid',
 'accountnumber',
 'accountratingcode',
 'address1_addressid',
 'address1_addresstypecode',
 'address1_city',
 'address1_composite',
 'address1_country',
 'address1_county',
 'address1_fax',
 'address1_freighttermscode',
 'address1_latitude',
 'address1_line1',
 'address1_line2',
 'address1_line3',
 'address1_longitude',
 'address1_name',
 'address1_postalcode',
 'address1_postofficebox',
 'address1_primarycontactname',
 'address1_shippingmethodcode',
 'address1_stateorprovince',
 'address1_telephone1',
 'address1_telephone2',
 'address1_telephone3',
 'address1_upszone',
 'address1_utcoffset',
 'address2_addressid',
 'address2_addresstypecode',
 'address2_city',
 'address2_composite',
 'address2_country',
 'address2_county',
 'address2_fax',
 'address2_freighttermscode',
 'address2_latitude',
 'address2_line1',
 'address2_line2',
 'address2_line3',
 'address2_longitude',
 'ad

# Nested data v2

In [136]:
def get_nested_keys(a):
    key_list=[]
    for i in a.keys():
        b=a[i]
        if "{" in str(b):
                key_list=key_list+[i]
    return(key_list)
sqlCtx.table("cxlvhlp_chatactivity").where("to_date(lastupdatedatetime)  = date_sub(current_date, 2)").select("entitystring").rdd.map(lambda x:x.asDict().values()[0].encode('utf-8')).map(lambda u:str(u).replace('null','None').replace('false','False').replace('true','True')).map(lambda y:ast.literal_eval(ast.literal_eval(y)['entitydata'])).map(lambda u: get_nested_keys(u)).filter(lambda x: len(x) > 0).collect()

[['activityadditionalparams'],
 ['activityadditionalparams'],
 ['activityadditionalparams'],
 ['activityadditionalparams'],
 ['activityadditionalparams'],
 ['activityadditionalparams'],
 ['activityadditionalparams'],
 ['activityadditionalparams'],
 ['activityadditionalparams']]

In [114]:
def get_nested_keys(a):
    key_list=[]
    for i in a.keys():
        b=a[i]
        if "{" in str(b):
                key_list=key_list+[i]
    return(key_list)
sqlCtx.table("annotation").where("message__id ='test14-16'").select("entitystring").rdd.map(lambda x:x.asDict().values()[0].encode('utf-8')).map(lambda u:str(u).replace('null','None').replace('false','False').replace('true','True')).map(lambda y:ast.literal_eval(ast.literal_eval(y)['entitydata'])).map(lambda u: get_nested_keys(u)).filter(lambda x: len(x) > 0).collect()

[['incident'], ['incident'], ['incident']]

# New fields v2

In [7]:
k="phonecall"
ignore_list=['entitylogicalname', 'SinkCreatedOn', 'SinkModifiedOn', 'messageid', 'sourcesystem', 'Id']
v1=sqlCtx.table(k).where("to_date(lastupdatedatetime)  = date_sub(current_date, 2)")
col_list=set(v1.columns)
v4=v1.select("entitystring").rdd.map(lambda x:x.asDict().values()[0].encode('utf-8')).map(lambda u:str(u).replace('null','None').replace('false','False').replace('true','True')).map(lambda y:ast.literal_eval(ast.literal_eval(y)['entitydata']).keys()+ast.literal_eval(y).keys()).map(lambda w:[x.lower() for x in w]).map(lambda x: set(x).difference(col_list)).collect()
v5=set([item for sublist in v4 for item in sublist]).difference(set(ignore_list))
if len(v5) >0:
                print("\\nTable: "+str(k)+" seem to get new Field(s) "+str(v5)+"\\n\\n")


\nTable: phonecall seem to get new Field(s) set(['activitytypecodename', 'isregularactivityname', 'gcct_visibleondealerportalname', 'isworkflowcreatedname', 'phonecall_rowversion', 'directioncodename', 'statecodename', 'prioritycodename', 'entitydata', 'gcct_Legacy_ModifiedOn', 'gcct_visibleoncustomerportalname', 'isbilledname', 'gcct_Legacy_ModifiedBy', 'gcct_DM_Source_Record_ID', 'statuscodename', 'gcct_Legacy_CreatedBy', 'leftvoicemailname', 'phonecall_identity'])\n\n


# New Fields added

In [66]:
k="account"
if k=="account":
            v1=sqlCtx.table(k)
            col_list=set(v1.columns)
            v2=v1.select("entitystring").rdd.map(lambda x: str(x).replace("Row(","").replace("u\'","").replace("\'","").replace("\\\\","").replace("\"","").replace("entitystring={entitydata:{","").replace("},entityid:",",entityid:").replace("})","").replace(r'{', '(').replace("}",")")).map(lambda r: re.sub(r'\(.+\)', '',r)).map(lambda z: re.findall(r',([a-zA-Z]+):', z)).map(lambda x: set(x).difference(col_list)).collect()
            v3=set([item for sublist in v2 for item in sublist])
            print k,v3

In [70]:
ignore_list=['entitylogicalname', 'SinkCreatedOn', 'SinkModifiedOn', 'messageid', 'sourcesystem', 'Id']
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").where("tableName not like '%dup%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            v1=sqlCtx.table(k).filter("to_date(lastupdatedatetime)  = date_sub(current_date, 1)")            
            col_list=set(v1.columns)
            v4=v1.select("entitystring").rdd.map(lambda x: str(x).replace("Row(","").replace("u\'","").replace("\'","").replace("\\\\","").replace("\"","").replace("entitystring={entitydata:{","").replace("},entityid:",",entityid:").replace("})","").replace(r'{', '(').replace("}",")")).map(lambda r: re.sub(r'\(.+\)', '',r)).map(lambda z: re.findall(r',([a-zA-Z]+):', z)).map(lambda x: set(x).difference(col_list)).collect()
            v5=set([item for sublist in v4 for item in sublist]).difference(set(ignore_list))
            if len(v5) >0:
                print("Table: "+str(k)+" seem to get new Field(s) "+str(v5))

Table: annotation seem to get new Field(s)set(['prefix', 'storagepointer', 'filepointer'])
Table: phonecall seem to get new Field(s)set(['activitytypecodename', 'isregularactivityname', 'isworkflowcreatedname', 'isbilledname', 'statecodename', 'prioritycodename', 'directioncodename', 'statuscodename', 'leftvoicemailname'])


In [225]:
def get_len_keys(a):
    key_list=[]
    b=ast.literal_eval(a.replace('null','None').replace('false','False').replace('true','True'))
    get_keys(b,key_list,"Main")
    return(key_list)



def get_keys(b,key_list,type_s):
    if(type_s=="Main"):
        for i in b.keys():
            try:
                key_list+ast.literal_eval(b[i]).keys()
                key_list+[i]
                get_keys(ast.literal_eval(b[i]),"sub")
            except:
                key_list+[i]
    elif(type_s=="sub"):
        for i in b.keys():
            try:
                key_list.extend(ast.literal_eval(b[i]).keys())
                #get_keys(ast.literal_eval(b[i]),"sub")
            except:
                pass
            
def get_keys_non_nested(a):
    key_list=[]
    b=ast.literal_eval(a.replace('null','None').replace('false','False').replace('true','True').encode('utf-8'))
    #for i in b.keys():
    #        try:
    #            key_list+ast.literal_eval(b[i].encode('utf-8')).keys()
    #        except:
    #            key_list+[i]
    key_list=b.keys()+ast.literal_eval(b["entitydata"]).keys()
    return(key_list)


def get_keys_nested(a):
    key_list=[]
    b=ast.literal_eval(a.replace('null','None').replace('false','False').replace('true','True').encode('utf-8'))
    for i in b.keys():
            try:
                c=ast.literal_eval(b[i])
                for r in c.keys():
                    if(len(ast.literal_eval(c)[r].keys()))>0:
                        key_list=key_list+[r]
            except:
                pass
    return(key_list)

def get_keys_nested2(a):
    key_list=[]
    b=ast.literal_eval(a.replace('null','None').replace('false','False').replace('true','True').encode('utf-8'))
    for i in ast.literal_eval(b["entitydata"]).keys():
           if "{" in ast.literal_eval(b["entitydata"])[i]:
                key_list+[i]
    return(key_list)


In [222]:
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType, IntegerType
udf1 = udf(lambda x:get_keys_nested2(x),StringType())
udf2 = udf(lambda x:get_keys_non_nested(x),StringType())

In [223]:
k="annotation"
if k=="annotation":
#for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bad%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            v1=sqlCtx.table(k).select("entitystring")
            df = v1.withColumn('New', udf1(v1.entitystring))
            

In [224]:
df.select("New").distinct().show(10,False)

Py4JJavaError: An error occurred while calling o4114.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 13 in stage 381.0 failed 4 times, most recent failure: Lost task 13.3 in stage 381.0 (TID 10638, hpchdd2r01n12.hpc.ford.com, executor 419): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/hdp/current/spark2-client/python/pyspark/worker.py", line 229, in main
    process()
  File "/usr/hdp/current/spark2-client/python/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/hdp/current/spark2-client/python/pyspark/worker.py", line 149, in <lambda>
    func = lambda _, it: map(mapper, it)
  File "<string>", line 1, in <lambda>
  File "/usr/hdp/current/spark2-client/python/pyspark/worker.py", line 74, in <lambda>
    return lambda *a: f(*a)
  File "<ipython-input-222-6490b2a01b01>", line 4, in <lambda>
  File "<ipython-input-221-ef39a28f7bb8>", li

In [226]:
k="annotation"
if k=="annotation":
#for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bad%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            v1=sqlCtx.table(k).select("entitystring")
            df = v1.withColumn('New', udf2(v1.entitystring))
            

In [227]:
df.select("New").distinct().show(10,False)

Py4JJavaError: An error occurred while calling o4180.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 20 in stage 383.0 failed 4 times, most recent failure: Lost task 20.3 in stage 383.0 (TID 10700, hpchdd2r01n14.hpc.ford.com, executor 427): ExecutorLostFailure (executor 427 exited caused by one of the running tasks) Reason: Container killed by YARN for exceeding memory limits. 2.1 GB of 2 GB physical memory used. Consider boosting spark.yarn.executor.memoryOverhead.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1599)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1587)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1586)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuf

In [230]:
sqlContext.table("account").select("entitystring").limit(2).rdd.map(lambda x:x.asDict().values()[0].encode('utf-8')).map(lambda u:str(u).replace('null','None').replace('false','False').replace('true','True'))..take(1)

SyntaxError: invalid syntax (<ipython-input-230-cda5994a82bd>, line 1)

In [15]:
ast.literal_eval('{"a":1,"b":{"c":4,"g":5},"f":7}')

{'a': 1, 'b': {'c': 4, 'g': 5}, 'f': 7}

In [32]:
re.sub(r'\(.+\)', '','{"a":1,"l":9,"b":("c":4,"d":g":5),"f":7}')

'{"a":1,"l":9,"b":,"f":7}'

In [ ]:
sqlCtx.table("annotation").select("entitystring").rdd.map(lambda x: str(x).replace("Row(","").replace("u\'","").replace("\'","").replace("\\\\","").replace("\"","")).map(lambda z: re.findall(r',([a-zA-Z]+):{', z)).filter(lambda x: len(x) > 0).collect()

In [14]:
#old stuff, dint work
def get_len_keys(a):
    key_list=[]
    b=ast.literal_eval(a.replace('null','None').replace('false','False').replace('true','True'))
    get_keys(b,key_list,"Main")
    #new_data_columns.extend(list(set(key_list).difference(set(col_list))))
    #return(new_data_columns)
    return(key_list)



def get_keys(b,key_list,type_s):
    if(type_s=="Main"):
        for i in b.keys():
            try:
                key_list.extend(ast.literal_eval(b[i]).keys())
                key_list.append(i)
                get_keys(ast.literal_eval(b[i]),"sub")
            except:
                key_list.append(i)
    elif(type_s=="sub"):
        for i in b.keys():
            try:
                key_list.extend(ast.literal_eval(b[i]).keys())
                get_keys(ast.literal_eval(b[i]),"sub")
            except:
                pass


import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType, IntegerType
udf1 = udf(lambda x:get_len_keys(x))

#for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").collect():
k="account"
if k=="account":
            k=m.asDict().values()[0].encode('utf-8')
            v1=sqlCtx.table(k).select("entitystring")
            global col_list2
            col_list=[]
            col_list.extend(sqlCtx.table(k).columns)
            col_list.extend(['entitylogicalname', 'eventtype', 'entityid', 'eventdatetime', 'sourcesystem', 'entitydata', 'messageid','shakey'])
            col_list2=[x.lower() for x in col_list]
            global new_data_columns
            new_data_columns=[]
            df = v1.withColumn('New', udf1(v1.entitystring))
            list1=list(df.select("New").distinct().toPandas()["New"])
            list2=set([i.lower() for sublist in list1 for i in sublist.encode('utf-8').replace("[","").replace("]","").split(", ")])
            #print k+"|"+",".join([str(i) for i in list2])
            print k+"|"+",".join([str(i) for i in set(list2).difference(col_list2)])+"|"+",".join([str(i) for i in set(col_list2).difference(list2)])       

database="dsc60230_gcct_tz_db"
for m in sqlContext.tables(database).select('tableName').where("tableName not like '%annotation%'").where("tableName not like '%temptz%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            v1=sqlCtx.table(k).where("lastupdatedatetime like '%2018-12-06%'").select("entitystring")
            global col_list2
            col_list=[]
            col_list.extend(sqlCtx.table(k).columns)
            col_list.extend(['entitylogicalname', 'eventtype', 'entityid', 'eventdatetime', 'sourcesystem', 'entitydata', 'messageid','shakey'])
            col_list2=[x.lower() for x in col_list]
            global new_data_columns
            new_data_columns=[]
            df = v1.withColumn('New', udf1(v1.entitystring))
            list1=list(df.select("New").distinct().toPandas()["New"])
            list2=set([i.lower() for sublist in list1 for i in sublist.encode('utf-8').replace("[","").replace("]","").split(", ")])
            list3=set(list2).difference(col_list2)
            list4=set(col_list2).difference(list2)
            if len(list3) > 0:
                print k+"|"+",".join([str(i) for i in list3])+"|"+",".join([str(i) for i in list4])

Old stuff

In [11]:
#old stuff, dint work
def get_len_keys(a):
    key_list=[]
    b=ast.literal_eval(a.replace('null','None').replace('false','False').replace('true','True'))
    get_keys(b,key_list,"Main")
    #new_data_columns.extend(list(set(key_list).difference(set(col_list))))
    #return(new_data_columns)
    return(key_list)



def get_keys(b,key_list,type_s):
    if(type_s=="Main"):
        for i in b.keys():
            try:
                key_list.extend(ast.literal_eval(b[i]).keys())
                key_list.append(i)
                get_keys(ast.literal_eval(b[i]),"sub")
            except:
                key_list.append(i)
    elif(type_s=="sub"):
        for i in b.keys():
            try:
                key_list.extend(ast.literal_eval(b[i]).keys())
                get_keys(ast.literal_eval(b[i]),"sub")
            except:
                pass


import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType, IntegerType
udf1 = udf(lambda x:get_len_keys(x))

#for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").collect():
k="account"
if k=="account":
            k=m.asDict().values()[0].encode('utf-8')
            v1=sqlCtx.table(k).select("entitystring")
            global col_list2
            col_list=[]
            col_list.extend(sqlCtx.table(k).columns)
            col_list.extend(['entitylogicalname', 'eventtype', 'entityid', 'eventdatetime', 'sourcesystem', 'entitydata', 'messageid','shakey'])
            col_list2=[x.lower() for x in col_list]
            global new_data_columns
            new_data_columns=[]
            df = v1.withColumn('New', udf1(v1.entitystring))
            list1=list(df.select("New").distinct().toPandas()["New"])
            list2=set([i.lower() for sublist in list1 for i in sublist.encode('utf-8').replace("[","").replace("]","").split(", ")])
            #print k+"|"+",".join([str(i) for i in list2])
            print k+"|"+",".join([str(i) for i in set(list2).difference(col_list2)])+"|"+",".join([str(i) for i in set(col_list2).difference(list2)])       


territory||entityimageid,gcct_territorygroup_entitytype,gcct_managerid,organizationid,gcct_manageridname,managerid_entitytype,sinkcreatedon,modifiedonbehalfbyname,createdonbehalfby,gcct_territorygroup,timezoneruleversionnumber,modifiedby,createdbyyominame,transactioncurrencyidname,messageid,createdonbehalfbyyominame,managerid,entityimage_timestamp,id,modifiedby_entitytype,overriddencreatedon,exchangerate,createdby_entitytype,gcct_facingregion,transactioncurrencyid,transactioncurrencyid_entitytype,eventdatetime,modifiedonbehalfby,createdby,shakey,sourcesystem,gcct_territorytypecode,gcct_isterritorygroup,utcconversiontimezonecode,organizationidname,entityimage_url,entitydata,description,lastupdatedatetime,createdbyname,gcct_zone,createdon,organizationid_entitytype,modifiedonbehalfbyyominame,modifiedbyyominame,createdonbehalfbyname,territoryid,gcct_region,gcct_csmtypecode,process__id,manageridname,message__id,name,modifiedonbehalfby_entitytype,gcct_manageridyominame,modifiedon,modifiedbyn

In [14]:
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bad%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            v1=sqlCtx.table(k).select("entitystring")
            df = v1.withColumn('New', udf1(v1.entitystring))
            g1=df.agg({"New": "max"}).collect()[0].asDict()['max(New)']
            g2=df.agg({"New": "min"}).collect()[0].asDict()['min(New)']
            g3=len(sqlCtx.table(k).columns)
            print("table:"+str(k)+",Max_extractable_columns:"+str(g1)+",Min_extractable_columns:"+str(g2)+",Columns_in_table:"+str(g3))

table:account,Max_extractable_columns:351,Min_extractable_columns:271,Columns_in_table:351
table:account_bkp,Max_extractable_columns:282,Min_extractable_columns:270,Columns_in_table:350
table:annotation,Max_extractable_columns:61,Min_extractable_columns:60,Columns_in_table:58
table:annotation_bkp,Max_extractable_columns:60,Min_extractable_columns:59,Columns_in_table:57
table:calendar,Max_extractable_columns:None,Min_extractable_columns:None,Columns_in_table:26
table:calendar_bkp,Max_extractable_columns:27,Min_extractable_columns:27,Columns_in_table:25
table:contact,Max_extractable_columns:363,Min_extractable_columns:351,Columns_in_table:410
table:contact_bkp,Max_extractable_columns:417,Min_extractable_columns:361,Columns_in_table:409
table:email,Max_extractable_columns:155,Min_extractable_columns:155,Columns_in_table:152
table:email_bkp,Max_extractable_columns:154,Min_extractable_columns:153,Columns_in_table:151
table:fax,Max_extractable_columns:None,Min_extractable_columns:None,Column

Run a query to get the counts of table

In [26]:

for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%bkp%'").where("tableName not like '%temptz%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            g1=sqlCtx.sql('select count(*) from '+k).select("count(1)").collect()[0].asDict().values()[0]
            print(str(k)+":"+str(g1))

account:83483
annotation:4939
bad_record:200480
calendar:0
contact:3613
email:110
fax:92
gcct_caseassignment:0
gcct_loyaltycostdetails:0
gcct_reactdata:0
gcct_recall:0
gcct_testdriverequest:0
incident:2
msdyn_question:0
phonecall:57790
systemuser:86


# Special characters

In [7]:
#Find special characters in the tables
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bad%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            g1=sqlCtx.sql('select count(*) from '+k+' where hex(entitystring) like "%0D%"').collect()
            print("Table:"+str(k)+":"+str(g1))

Table:account:[Row(count(1)=0)]
Table:annotation:[Row(count(1)=0)]
Table:calendar:[Row(count(1)=0)]
Table:contact:[Row(count(1)=0)]
Table:email:[Row(count(1)=0)]
Table:fax:[Row(count(1)=0)]
Table:gcct_caseassignment:[Row(count(1)=0)]
Table:gcct_loyaltycostdetails:[Row(count(1)=0)]
Table:gcct_reactdata:[Row(count(1)=0)]
Table:gcct_recall:[Row(count(1)=0)]
Table:gcct_testdriverequest:[Row(count(1)=0)]
Table:incident:[Row(count(1)=0)]
Table:phonecall:[Row(count(1)=0)]
Table:systemuser:[Row(count(1)=0)]


In [8]:
#Find special characters in the tables
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bad%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            g1=sqlCtx.sql('select count(*) from '+k+' where hex(entitystring) like "%C2A0%"').collect()
            print("Table:"+str(k)+":"+str(g1))

Table:account:[Row(count(1)=0)]
Table:annotation:[Row(count(1)=0)]
Table:calendar:[Row(count(1)=0)]
Table:contact:[Row(count(1)=2)]
Table:email:[Row(count(1)=1124)]
Table:fax:[Row(count(1)=0)]
Table:gcct_caseassignment:[Row(count(1)=0)]
Table:gcct_loyaltycostdetails:[Row(count(1)=0)]
Table:gcct_reactdata:[Row(count(1)=0)]
Table:gcct_recall:[Row(count(1)=0)]
Table:gcct_testdriverequest:[Row(count(1)=0)]
Table:incident:[Row(count(1)=0)]
Table:phonecall:[Row(count(1)=36)]
Table:systemuser:[Row(count(1)=0)]


# Duplicates

In [14]:
#find duplicates in data
#,id,lastupdatedatetime,
sqlCtx.sql('select fullname,shakey,count(*) from contact group by fullname,shakey having count(*)>1 ').show(1000,False)

+--------+------+--------+
|fullname|shakey|count(1)|
+--------+------+--------+
+--------+------+--------+



In [22]:
#find duplicates in data
#,id,lastupdatedatetime,
sqlCtx.sql('select * from contact where fullname like "Ford_Test Customer_18122018234339"').show(1000,False)

+----------------+------------------+----+-------------+--------------+---------+----------+-----------------------+---------------+----------------+-------------------------+----------+------------------------+------------------+-------------------------+----------------------------+---------------+-------------------+-------------------------+----------------+---------------+-------------------------+---------------------------+----------------+-----------------------+--------------------------+--------------+---------------------------+---------------------------+----------------+------------------------+---------------------------+-------------+------------------------+----------------+-------------+---------------------------+-----------------------+--------------------+----------------------+-----------------------------+-------------------+---------------+-------------------+----------------------------+---------------------------------+-------------+------------+---------------

In [12]:
#find duplicates in data
sqlCtx.sql('select id,shakey,lastupdatedatetime,process__id,count(*) from account group by id,shakey,lastupdatedatetime,process__id having count(*) >1 ').show(100,False)

+----+----------------------------------------------------------------+-------------------+-------------------+--------+
|id  |shakey                                                          |lastupdatedatetime |process__id        |count(1)|
+----+----------------------------------------------------------------+-------------------+-------------------+--------+
|null|0412abd613d6b062b5ad77e6b7ef34dbd6f3d02b612d10535279fb5e9951f2ee|2018-12-18 15:04:12|PN_1270803221609367|3       |
+----+----------------------------------------------------------------+-------------------+-------------------+--------+



In [6]:
# on 01-09-2019
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bad%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            stmnt='select shakey,count(*) from '+k+' group by shakey having count(*) >1 '
            a=sqlCtx.sql(stmnt)
            if a.count()>0:
                print("Table: "+str(k)+" has "+str(a.count())+" duplicate(s)")

Table: account has 1 duplicate(s)
Table: activityparty has 1 duplicate(s)
Table: appointment has 1 duplicate(s)
Table: contact has 1 duplicate(s)
Table: email has 1 duplicate(s)
Table: gcct_buybackevaluationmilestones has 1 duplicate(s)
Table: gcct_caseassignment has 1 duplicate(s)
Table: gcct_genericinformation has 1 duplicate(s)


In [5]:
#find duplicates in data
#,id,lastupdatedatetime,
tab_list_dub=['account','activityparty','appointment','contact','email' ,'gcct_buybackevaluationmilestones' ,'gcct_caseassignment' ,'gcct_genericinformation' ]
for tab in tab_list_dub:
     sqlCtx.sql('select * from '+tab+' where shakey in (select shakey from '+tab+' group by shakey having count(*)>1) ').show(1000,False)

+----------------+------------------+----+-------------+--------------+---------+----------+----------------+-------------------------+------------------------+-------------------------+------------------+----------------------------+-------------------+-------------------------+-------------------+----------------+----------------+------------+--------------------+---------------------------+---------------------------+----------------+--------------------------+-----------------+----------------+------------------------+---------------------------+---------------+-------------+-------------+------------------+---------------+--------------------+-----------------------------+-------------------+---------------+---------------------+----------------------------+-------------------+---------------+---------------------------+------------+------------------------+-----------------------------+-----------+--------------+-----------+-------------------+------------------------------+-----

+----------------------+------------------------+----+-------------+--------------+---------------------+----------------+---------------+--------+----------+----------+--------------+--------------+-------------------------+------------------------------------+------------------+------------------------------------+---------------------+------------------------------------+------------------+----------+---------------+-----------+-----------+--------------+------------------+-------------+----------------------------+-----------+------------------+------------------------------------+------+------------+-------------------+----------+-------------------+----------------------------------------------------------------+-------------------+---------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

+--------------------+----------------------+----+-------------+--------------+---------+----------+------------+----------------+----------------+--------+-----------------+-----------------+-------------------+---------------+-----------------+-------+-------------+------------------+-------------+----------------------------+-----------------+----------------------------+----------+---------------------+------------------------------------+----------------------------+------------------------------------+---------------------------------------+------------------+-----------------------------+------------------+-----------------------------+------------------+-----------------------------+------------------------------------+-----------------------+------------------------------------+--------------------------------+------------------------------------+----------------+------------------------------------+--------------------+------------------------------------+--------------------

+----------------+------------------+----+-------------+--------------+---------+----------+-----------------------+---------------+----------------+-------------------------+----------+------------------------+------------------+-------------------------+----------------------------+---------------+-------------------+-------------------------+----------------+---------------+-------------------------+---------------------------+----------------+-----------------------+--------------------------+--------------+---------------------------+---------------------------+----------------+------------------------+---------------------------+-------------+------------------------+----------------+-------------+---------------------------+-----------------------+--------------------+----------------------+-----------------------------+-------------------+---------------+-------------------+----------------------------+---------------------------------+-------------+------------+---------------

+--------------+----------------+----+-------------+--------------+---------+----------+-------------+-------------------+-----------------+--------------------+------------+-----------------+----------------------------+-----------------+-------------------------+--------------------+--------+---------------+-----------------+--------------------------+------------------------+------------------+----------+-------------+-----+----------------+----------------+---------------------------+-----------------+----------------------------+-----------+----------------------+---------+--------------------+------------------------------------+-----------------------------+--------------+-------------------------+----------+---------------------+------------------------------------+---------------------+------------------+-----------------------------+----------+---------------------+------------------------------------+---------------------+------------------------------------+----------------

+----+-------------+--------------+---------+----------+---------+----------------+------------------+----------------+----------------+--------------------+------------+-----------------+--------+-----------------+----------------------------------------------+-------------+-----------------------------------+-----------------------------+-------------+-------------------------+-------------------+----------------------------+-----------------------------+-----+----------------+------------------------------------+-------------------------+------------------------------------+-----------------------------+------------------------------------+-------------------------+------------------------------------+------------------------+------------------------------------+------------------------+-----------------+----------------------------+-------------------+------------------------------+------------------------------------+---------------------+------------------+-----------------------

+----------------------------+------------------------------+----+-------------+--------------+---------+----------+----------------+------------+--------------------+---------+--------+-----------------------------+-----------------+-------------+--------------------+---------------------------+-------------+-----------------+------------------------------------+----------------------------+------------------------------------+-----------------------------+------------------------------------+---------------------+---------------+--------------------------+------------------------------------+----------------------------+------------------------------------+----------------+---------------------+--------------------------------+------------------------------------+-----------------------------------+------------------------------------+-----------------------+------------------------------------+--------------------+------------------------------------+---------------------+---------+

+----+-------------+--------------+---------+----------+----------------+------------+--------------------+---------+--------+-----------------+-------------+-------------+-----------------+------------------------------------+----------------------+-----------------+----------------------------+------------------+-----------------------------+------------------------------------+---------------------+---------------+--------------------------+------------------------------------+----------------------------+-----+----------------+------------------------------------+--------------------------------+------------+-----------------------+------------------------------------+--------------------+------------------------------------+---------------------+---------+--------------------+----------+---------------------+------------------------------------+-----------------------------+------------------------------------+------------------+----+---------+---------+--------+----+----+--------

In [57]:
sqlContext.table('account').where(col("lastupdatedatetime").like("%2018-09-26%")).groupby("id","shakey").count().filter(col('count')>1).show(10,False)

+------------------------------------+----------------------------------------------------------------+-----+
|id                                  |shakey                                                          |count|
+------------------------------------+----------------------------------------------------------------+-----+
|8190E634-4284-E811-A83D-000D3A37C740|a240c5e3d8a0ae3f7d0e4d31852d5a757b65cfc676d542ec13686a92e007bbf0|2    |
+------------------------------------+----------------------------------------------------------------+-----+



In [58]:
sqlContext.table('account').where(col("lastupdatedatetime").like("%2018-09-26%")).filter(col('shakey')=='a240c5e3d8a0ae3f7d0e4d31852d5a757b65cfc676d542ec13686a92e007bbf0').show(10,False)

+----------------+------------------+------------------------------------+-------------------+-------------------+---------+----------+------------------------+---------------------------+------------------+---------------+-------------------------+-----------------+----------------------------+-------------------------+----------------+--------------------------+-------------+------------------------+----------------+------------------+----------------+---------------+----------------+-------------------+------------+---------------------------+---------------------------+-------------------------+-------------+---------------+------------+-------------------+------------------------------+------------------------+--------------+--------+----------+-------------------+-------------+-----------+------+----------+-----------+----------------------+---------+---------------------+--------------------------------+---------------------------+--------------------------------------+---------

# Yao

Searching for string columns of large lengths

In [3]:
#Yao
#find the min and max length of string columns
for m in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bad%'").where("tableName like '%bkp%'").collect():
        k=m.asDict().values()[0].encode('utf-8')
        for b in sqlCtx.sql('desc '+k).filter(col("data_type") == "string").filter(col("col_name") != "entitystring").select("col_name").collect():
                c=b.asDict().values()[0].encode('utf-8')
                s=sqlContext.sql('select max(length('+c+')),min(length('+c+')) from dsc60230_gcct_tz_db.'+k).collect()
                if str(s[0].asDict().values()[1]) != "None":
                    print(k+','+c+','+str(s[0].asDict().values()))

account_bkp,account_rowversion,[12, 12]
account_bkp,id,[36, 36]
account_bkp,gcct_dealerservicemanagerid,[36, 36]
account_bkp,gcct_dealerservicemanagerid_entitytype,[7, 7]
account_bkp,owningbusinessunit,[36, 36]
account_bkp,owningbusinessunit_entitytype,[12, 12]
account_bkp,territoryid,[36, 36]
account_bkp,territoryid_entitytype,[9, 9]
account_bkp,primarycontactid,[36, 36]
account_bkp,primarycontactid_entitytype,[7, 7]
account_bkp,gcct_dealercontact,[36, 36]
account_bkp,gcct_dealercontact_entitytype,[7, 7]
account_bkp,modifiedby,[36, 36]
account_bkp,modifiedby_entitytype,[10, 10]
account_bkp,modifiedonbehalfby,[36, 36]
account_bkp,modifiedonbehalfby_entitytype,[10, 10]
account_bkp,gcct_region,[36, 36]
account_bkp,gcct_region_entitytype,[11, 11]
account_bkp,owninguser,[36, 36]
account_bkp,owninguser_entitytype,[10, 10]
account_bkp,owningteam,[36, 36]
account_bkp,owningteam_entitytype,[4, 4]
account_bkp,transactioncurrencyid,[36, 36]
account_bkp,transactioncurrencyid_entitytype,[19, 19]
a

contact_bkp,gcct_stag2,[7, 7]
contact_bkp,gcct_ltag1,[7, 7]
contact_bkp,emailaddress3,[38, 7]
contact_bkp,emailaddress2,[39, 7]
contact_bkp,emailaddress1,[2, 76]
contact_bkp,nickname,[7, 3]
contact_bkp,lastname,[50, 1]
contact_bkp,telephone1,[0, 14]
contact_bkp,adx_identity_passwordhash,[68, 68]
contact_bkp,gcct_stag1,[7, 7]
contact_bkp,address2_addressid,[36, 36]
contact_bkp,address1_composite,[1, 149]
contact_bkp,msdyusd_currentprofile,[4, 4]
contact_bkp,owneridyominame,[25, 8]
contact_bkp,address1_line1,[100, 1]
contact_bkp,mastercontactidyominame,[9, 14]
contact_bkp,address1_country,[2, 32]
contact_bkp,firstname,[47, 1]
contact_bkp,suffix,[10, 2]
contact_bkp,gcct_contactpreferenceupdatebyuseridyominame,[25, 9]
contact_bkp,parentcustomeridyominame,[16, 7]
contact_bkp,msdyusd_facebook,[8, 8]
contact_bkp,contactid,[36, 36]
contact_bkp,address1_line2,[1, 39]
contact_bkp,gcct_surveysuppressionsetbyuseridyominame,[9, 25]
contact_bkp,modifiedbyyominame,[25, 9]
contact_bkp,gcct_datasources

gcct_loyaltycostdetails_bkp,modifiedby_entitytype,[10, 10]
gcct_loyaltycostdetails_bkp,owninguser,[36, 36]
gcct_loyaltycostdetails_bkp,owninguser_entitytype,[10, 10]
gcct_loyaltycostdetails_bkp,owningbusinessunit,[36, 36]
gcct_loyaltycostdetails_bkp,owningbusinessunit_entitytype,[12, 12]
gcct_loyaltycostdetails_bkp,ownerid,[36, 36]
gcct_loyaltycostdetails_bkp,ownerid_entitytype,[4, 10]
gcct_loyaltycostdetails_bkp,gcct_name,[41, 77]
gcct_loyaltycostdetails_bkp,modifiedbyyominame,[20, 9]
gcct_loyaltycostdetails_bkp,gcct_agentnameidyominame,[9, 9]
gcct_loyaltycostdetails_bkp,gcct_rolinenumber,[6, 1]
gcct_loyaltycostdetails_bkp,gcct_rejectiondetails,[20, 20]
gcct_loyaltycostdetails_bkp,gcct_approvalrejectioncode,[4, 4]
gcct_loyaltycostdetails_bkp,owneridyominame,[12, 9]
gcct_loyaltycostdetails_bkp,gcct_loyaltycostdetailsid,[36, 36]
gcct_loyaltycostdetails_bkp,createdbyyominame,[9, 12]
gcct_loyaltycostdetails_bkp,modifiedonbehalfbyyominame,[12, 9]
gcct_loyaltycostdetails_bkp,lastupdtby,[8, 

incident_bkp,owninguser,[36, 36]
incident_bkp,owninguser_entitytype,[10, 10]
incident_bkp,slainvokedid,[36, 36]
incident_bkp,msa_partnercontactid,[36, 36]
incident_bkp,msa_partnercontactid_entitytype,[7, 7]
incident_bkp,gcct_customercountryid,[36, 36]
incident_bkp,gcct_customercountryid_entitytype,[12, 12]
incident_bkp,gcct_surveysenttocontact,[36, 36]
incident_bkp,gcct_surveysenttocontact_entitytype,[7, 7]
incident_bkp,gcct_lincolncsm,[36, 36]
incident_bkp,gcct_lincolncsm_entitytype,[10, 10]
incident_bkp,gcct_closedby,[36, 36]
incident_bkp,gcct_closedby_entitytype,[10, 10]
incident_bkp,modifiedonbehalfby,[36, 36]
incident_bkp,modifiedonbehalfby_entitytype,[10, 10]
incident_bkp,primarycontactid,[36, 36]
incident_bkp,primarycontactid_entitytype,[7, 7]
incident_bkp,gcct_reopenedby,[36, 36]
incident_bkp,gcct_reopenedby_entitytype,[10, 10]
incident_bkp,gcct_fmcccontract,[36, 36]
incident_bkp,gcct_fmcccontract_entitytype,[17, 17]
incident_bkp,parentcaseid,[36, 36]
incident_bkp,parentcaseid_

systemuser_bkp,windowsliveid,[37, 12]
systemuser_bkp,address1_postalcode,[5, 10]
systemuser_bkp,address1_composite,[33, 81]
systemuser_bkp,address2_addressid,[36, 36]
systemuser_bkp,address1_fax,[13, 13]
systemuser_bkp,createdbyyominame,[15, 15]
systemuser_bkp,fullname,[6, 45]
systemuser_bkp,modifiedonbehalfbyyominame,[25, 8]
systemuser_bkp,defaultodbfoldername,[11, 3]
systemuser_bkp,lastupdtby,[8, 8]
systemuser_bkp,shakey,[64, 64]
systemuser_bkp,oozieid,[36, 36]


# Testing

We have 22 test cases and expected results. the following code helps us with it 

In [12]:
sqlContext.table('account').where("message__id like '%est%'").select("message__id","process__id","lastupdatedatetime","entityid").show(1000,False)

+-----------+--------------------+-------------------+--------+
|message__id|process__id         |lastupdatedatetime |entityid|
+-----------+--------------------+-------------------+--------+
|Test1      |PN_31244864526122068|2019-01-29 13:44:47|Test1   |
+-----------+--------------------+-------------------+--------+



In [12]:
sqlContext.table('account').where("process__id = 'PN_31244864526122068'").where("message__id like '%est%'").select("message__id","entitystring").show(1000,False)

+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
sqlContext.table('incident').where("message__id like '%est%'").select("message__id","process__id","lastupdatedatetime","entityid").show(1000,False)

+-----------+--------------------+-------------------+------------------------------------+
|message__id|process__id         |lastupdatedatetime |entityid                            |
+-----------+--------------------+-------------------+------------------------------------+
|Test7      |PN_31244864526122068|2019-01-29 13:44:47|96532B43-05F8-E811-A970-000D3A34A0AA|
+-----------+--------------------+-------------------+------------------------------------+



In [15]:
sqlContext.table('annotation').where("message__id like '%est%'").select("eventtype","message__id","process__id","lastupdatedatetime","entityid").show(1000,False)

+---------+-----------+--------------------+-------------------+-------------------------------------+
|eventtype|message__id|process__id         |lastupdatedatetime |entityid                             |
+---------+-----------+--------------------+-------------------+-------------------------------------+
|create   |test11     |PN_31244864526122068|2019-01-29 13:44:47|38Bfs44BB-81F2-E811-A96D-000D3A328903|
|update   |test14-16  |PN_31244864526122068|2019-01-29 13:44:47|38BA44BB-81F2-E811-A96D-000D3A328903 |
|         |test20     |PN_31244864526122068|2019-01-29 13:44:47|38BA44BB-81F2-E811-A96D-000D3A328903 |
|delete   |test17     |PN_31244864526122068|2019-01-29 13:44:47|38BA44BB-81F2-E811-A96D-000D3A328903 |
|create   |test14-16  |PN_31244864526122068|2019-01-29 13:44:47|38BA44BB-81F2-E811-A96D-000D3A328903 |
|delete   |test14-16  |PN_31244864526122068|2019-01-29 13:44:47|38BA44BB-81F2-E811-A96D-000D3A328903 |
|create   |test13     |PN_31244864526122068|2019-01-29 13:44:47|38BA44BB-

In [18]:
#sqlContext.table('bad_record').where("process__id like 'PN_31244864526122068'").select("data","message__id","process__id").show(1000,False)
sqlContext.table('bad_record').where("process__id like 'PN_31244864526122068'").show(1000,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
sqlContext.table('bad_record').where("to_date(lastupdatedatetime) like '2019-01-%'").where("data like '%Test8%'").show(1000,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
sqlContext.table('bad_record').where("data like '%Nani%'").show(1000,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
sqlCtx.sql('select data,exception from bad_record where reason like "Error Parsing test22%"').show(1000,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


# Testing with Srini 

In [35]:
sqlCtx.sql("select fullname,gcct_sourcesystemid,telephone2,address1_line1,address1_postalcode,address1_stateorprovince from contact where fullname like '%Nani%'").show(100,False)

+--------+-------------------+----------+--------------+-------------------+------------------------+
|fullname|gcct_sourcesystemid|telephone2|address1_line1|address1_postalcode|address1_stateorprovince|
+--------+-------------------+----------+--------------+-------------------+------------------------+
+--------+-------------------+----------+--------------+-------------------+------------------------+



In [36]:
#to_date(lastupdatedatetime) >"2019-01-15" and#.filter("date2 like '2019-02%'")
sqlCtx.sql('select data,datatype,to_date(lastupdatedatetime) as date2,reason,exception,lastupdatedatetime from bad_record where  exception like "%Datatype discrepancy%"').filter("datatype ='account'").filter("lastupdatedatetime like '2019-02%'").show(10,False)

+----+--------+-----+------+---------+------------------+
|data|datatype|date2|reason|exception|lastupdatedatetime|
+----+--------+-----+------+---------+------------------+
+----+--------+-----+------+---------+------------------+



In [32]:
#to_date(lastupdatedatetime) >"2019-01-15" and
sqlCtx.table('gcct_warrantyloanerrequest').filter("lastupdatedatetime like '2019-02%'").filter("gcct_bestdaytimephone is not NULL").show(10,False)

+----------------+------------------+------------------------------------+-------------------+-------------------+---------+----------+----------------+-------------------------+------------------------+-------------------------+---------------------+------------------+----------------------------+-------------------+-------------------------+-------------------+----------------+----------------+------------+--------------------+---------------------------+---------------------------+----------------+--------------------------+-----------------+----------------+------------------------+---------------------------+---------------+-------------+-------------+------------------+---------------+--------------------+-----------------------------+-------------------+---------------+---------------------+-----------------------------------+----------------------------+-------------------+---------------+-----------------------------------+---------------------------+------------+-------------

In [ ]:
#to_date(lastupdatedatetime) >"2019-01-15" and
sqlCtx.table('gcct_warrantyloanerrequest').filter("to_date(lastupdatedatetime) like '2019-02%'").show(10,False)

In [12]:
sqlContext.sql("select * from bad_record where datatype = 'gcct_loyaltyassistance' and exception like '%Datatype discrepancy%'").show(10,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
sqlContext.sql("select * from gcct_loyaltyassistance").show(10,False)

+-------------------------------+---------------------------------+------------------------------------+-------------------+-------------------+---------+----------+--------------------+-------------------+---------------------+---------+-----------------+-------------------+-------------+-----------------+----------------+------------+----------------------------+-----------------+-------------------------+----------------------------------------+--------+-----------------+---------------------------+----------------------------------+--------------------------+-------------+------------------------------------+----------------------------+-------------+---------------------------------------------+-----------------+----------------------------------------------+---------------------------------+--------------------------------------+-----------------------------------------+-----+----------------+------------------------------------+----------------------+----------------------------

In [104]:
#,address1_line1,address1_postalcode,address1_stateorprovince
sqlCtx.sql("select id from contact where fullname like '%Nani%'").show(1000,False)

+---+
|id |
+---+
+---+



In [118]:
sqlCtx.sql("select title,workeridyominame from queueitem where title like 'CAS-25782-Q4C8F2'").show(10,False)

+----------------+----------------+
|title           |workeridyominame|
+----------------+----------------+
|CAS-25782-Q4C8F2|Srinivasa Gadde |
+----------------+----------------+



In [5]:
sqlCtx.sql("select * from gcct_caseassignment where subject like '%CAS-25782-Q4C8F2%'").show(10,False)

+----------------------------+------------------------------+------------------------------------+-------------------+-------------------+---------+----------+----------------+------------+--------------------+---------+--------+----------------+-----------------------------+-----------------+-------------+--------------------+---------------------------+-------------+-----------------+------------------------------------+----------------------------+------------------------------------+-----------------------------+------------------------------------+---------------------+---------------+--------------------------+------------------------------------+----------------------------+------------------------------------+----------------+---------------------+--------------------------------+------------------------------------+-----------------------------------+------------------------------------+-----------------------+------------------------------------+--------------------+---------

In [126]:
sqlCtx.sql("select customeridyominame,gcct_caseclassificationlevel4name,gcct_caseclassificationlevel1name,gcct_caseclassificationlevel2name,gcct_caseclassificationlevel3name from incident where entitystring like 'CAS-25782-Q4C8F2'").show(10,False)

+------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+
|customeridyominame|gcct_caseclassificationlevel4name|gcct_caseclassificationlevel1name|gcct_caseclassificationlevel2name|gcct_caseclassificationlevel3name|
+------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+
+------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+



# Get data counts of all timestamp columns in database

Dont try this GCCT has too many timestamps

In [1]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import to_date
import pyspark.sql.functions 

for tab2 in sqlContext.tables("dsc60230_gcct_tz_db").select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bad%'").where("tableName like '%bkp%'").collect():
    tab=tab2.asDict().values()[0].encode('utf-8')
    a=sqlContext.sql("desc formatted "+tab).filter(col("data_type") == 'timestamp')
    if a.count()>0:
        for timestamp_cols in a.select("col_name").rdd.flatMap(list).collect():
             sqlContext.table(tab).withColumn(timestamp_cols+"New", to_date(col(timestamp_cols))).groupby(col(timestamp_cols+"New")).count().sort(col(timestamp_cols+"New").desc()).withColumn("table",lit(tab)).show(10,False)
        

KeyboardInterrupt: 